In [ ]:
from pyspark.sql import Row

def mapper(line):
    fields = line.split(',')
    return Row(
        ID = int(fields[0]),
        name = fields[1],        # No need to manually encode here
        age = int(fields[2]),
        numberfriends = int(fields[3])
    )

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("SQL_on_DataFrame") \
        .getOrCreate()

In [ ]:
columns = ["EmpID", "Name", "Department", "Salary", "Age"]
data = [
    (101, "Amit Sharma",   "IT",        55000,  29),
    (102, "Riya Verma",    "HR",        48000,  31),
    (103, "Karan Gupta",   "Finance",   62000,  26),
    (104, "Sneha Patel",   "IT",        75000,  36),
    (105, "Manoj Singh",   "Marketing", 52000,  28),
    (106, "Priya Nair",    "Finance",   68000,  41),
    (107, "Rahul Mehta",   "IT",        83000,  38),
    (108, "Tina Joseph",   "HR",        46000,  24)
]
df = spark.createDataFrame(data, columns)
df.show()

+-----+-----------+----------+------+---+
|EmpID|       Name|Department|Salary|Age|
+-----+-----------+----------+------+---+
|  101|Amit Sharma|        IT| 55000| 29|
|  102| Riya Verma|        HR| 48000| 31|
|  103|Karan Gupta|   Finance| 62000| 26|
|  104|Sneha Patel|        IT| 75000| 36|
|  105|Manoj Singh| Marketing| 52000| 28|
|  106| Priya Nair|   Finance| 68000| 41|
|  107|Rahul Mehta|        IT| 83000| 38|
|  108|Tina Joseph|        HR| 46000| 24|
+-----+-----------+----------+------+---+



In [ ]:
# Register DataFrame as SQL Table 2nd technics
# df.createOrReplaceTempView("friend")

In [ ]:
spark.sql("SELECT * FROM friend").show()

+-----+-----------+----------+------+---+
|EmpID|       Name|Department|Salary|Age|
+-----+-----------+----------+------+---+
|  101|Amit Sharma|        IT| 55000| 29|
|  102| Riya Verma|        HR| 48000| 31|
|  103|Karan Gupta|   Finance| 62000| 26|
|  104|Sneha Patel|        IT| 75000| 36|
|  105|Manoj Singh| Marketing| 52000| 28|
|  106| Priya Nair|   Finance| 68000| 41|
|  107|Rahul Mehta|        IT| 83000| 38|
|  108|Tina Joseph|        HR| 46000| 24|
+-----+-----------+----------+------+---+



In [ ]:
print(df)

DataFrame[EmpID: bigint, Name: string, Department: string, Salary: bigint, Age: bigint]


In [ ]:
#Select specific columns
spark.sql("SELECT Name, Salary FROM  friend").show()


+-----------+------+
|       Name|Salary|
+-----------+------+
|Amit Sharma| 55000|
| Riya Verma| 48000|
|Karan Gupta| 62000|
|Sneha Patel| 75000|
|Manoj Singh| 52000|
| Priya Nair| 68000|
|Rahul Mehta| 83000|
|Tina Joseph| 46000|
+-----------+------+



In [ ]:
# use filter on age
spark.sql("select age from friend Where age>30") .show()

+---+
|age|
+---+
| 31|
| 36|
| 41|
| 38|
+---+



In [ ]:
# use Order(ASC DESC) friend by salary
spark.sql("select * from friend order By salary DESC").show()

+-----+-----------+----------+------+---+
|EmpID|       Name|Department|Salary|Age|
+-----+-----------+----------+------+---+
|  107|Rahul Mehta|        IT| 83000| 38|
|  104|Sneha Patel|        IT| 75000| 36|
|  106| Priya Nair|   Finance| 68000| 41|
|  103|Karan Gupta|   Finance| 62000| 26|
|  101|Amit Sharma|        IT| 55000| 29|
|  105|Manoj Singh| Marketing| 52000| 28|
|  102| Riya Verma|        HR| 48000| 31|
|  108|Tina Joseph|        HR| 46000| 24|
+-----+-----------+----------+------+---+



In [ ]:

# Group by Department and find average salary
spark.sql("""select department, avg (salary) avgsalary
             from friend
             group By  department """).show()

+----------+---------+
|department|avgsalary|
+----------+---------+
|        HR|  47000.0|
|   Finance|  65000.0|
|        IT|  71000.0|
| Marketing|  52000.0|
+----------+---------+



In [ ]:
#SQL-Style Functions in DataFrame API
from pyspark.sql import functions as F

In [ ]:
df.filter(df.Age > 30).show()

+-----+-----------+----------+------+---+
|EmpID|       Name|Department|Salary|Age|
+-----+-----------+----------+------+---+
|  102| Riya Verma|        HR| 48000| 31|
|  104|Sneha Patel|        IT| 75000| 36|
|  106| Priya Nair|   Finance| 68000| 41|
|  107|Rahul Mehta|        IT| 83000| 38|
+-----+-----------+----------+------+---+



In [ ]:
df.select("name","salary").show()

+-----------+------+
|       name|salary|
+-----------+------+
|Amit Sharma| 55000|
| Riya Verma| 48000|
|Karan Gupta| 62000|
|Sneha Patel| 75000|
|Manoj Singh| 52000|
| Priya Nair| 68000|
|Rahul Mehta| 83000|
|Tina Joseph| 46000|
+-----------+------+



In [ ]:
df.groupBy("Department").agg(F.avg("Salary").alias("avgsalary")) .show()


+----------+---------+
|Department|avgsalary|
+----------+---------+
|        HR|  47000.0|
|   Finance|  65000.0|
|        IT|  71000.0|
| Marketing|  52000.0|
+----------+---------+



In [ ]:
df.agg(F.max("Salary").alias("MaxSalary")).show()

+---------+
|MaxSalary|
+---------+
|    83000|
+---------+



In [ ]:
df.groupBy("Department") \
  .agg(F.avg("Salary").alias("AvgSalary")) \
  .filter("AvgSalary > 60000") \
  .show()

In [ ]:
df.withColumn(
    "SalaryLevel",
    F.when(df.Salary > 70000, "High")
     .when(df.Salary >= 50000, "Medium")
     .otherwise("Low")
).show()

In [ ]:
df.filter((df.Department == "IT") & (df.Age.between(30, 40))).show()